In [1]:
%pip install bertviz
%pip install jupyterlab
%pip install ipywidgets
%pip install -U accelerate
%pip install -U transformers
%pip install torch
%pip install datasets
%pip install numpy

import accelerate
print(accelerate.__version__)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, utils
from bertviz import head_view, model_view
import torch
utils.logging.set_verbosity_error()  # Suppress standard warnings

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load saved weights
model_path = "../../output/job-5193/selected_layers_state_dict.pth"  # Path to saved model weights
saved_layers = torch.load(model_path, map_location=device) # Load saved weights
print("Loaded saved layers.\nSaved layers keys:",saved_layers.keys())  # Display saved layers

# Load model and tokenizer
model_name = "bert-base-uncased"  # Find popular HuggingFace models here: https://huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=28, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load saved weights into model
model_type = "head+1"
if model_type == "head+1":
    model.classifier.load_state_dict(saved_layers["classifier"])
    model.bert.encoder.layer[11].load_state_dict(saved_layers["encoder"])
    model.bert.pooler.load_state_dict(saved_layers["pooler"])

/var/folders/fj/3dl8rf214jqcw2z2kdlt5b8h0000gt/T/ipykernel_56762/1916512824.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_layers = torch.load(model_path, map_lo

Loaded saved layers.
Saved layers keys: dict_keys(['classifier', 'encoder', 'pooler'])


In [18]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding, Trainer

dataset = load_dataset("google-research-datasets/go_emotions")

print("Original dataset:\n", dataset)

# Define a filtering function to keep only examples with a single label
def filter_single_label(example):
    return len(example["labels"]) == 1


# Apply the filter to all splits (train, validation, test)
filtered_dataset = dataset.filter(filter_single_label)

print("Filtered dataset:\n", filtered_dataset)


# Step 2: Preprocess the dataset (tokenize)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="longest")


# Tokenize the dataset
tokenized_datasets = filtered_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

# Setup training


Original dataset:
 DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})
Filtered dataset:
 DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 36308
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 4548
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 4590
    })
})


Map:   0%|          | 0/36308 [00:00<?, ? examples/s]

Map:   0%|          | 0/4548 [00:00<?, ? examples/s]

Map:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [20]:
print(tokenized_datasets["test"]["text"][1615])

Thank you for your service


In [21]:
correct_instances = [[1615, "gratitude"], [3174, "admiration"], [23, "curiosity"]]
incorrect_instances = [[745, "desire"], [1863, "disapproval"], [1150, "curiosity"]]

In [22]:
# If you have a label map (e.g., emotions or sentiments), you can map the index to the label
label_map = {
    "0": "admiration",
    "1": "amusement",
    "2": "anger",
    "3": "annoyance",
    "4": "approval",
    "5": "caring",
    "6": "confusion",
    "7": "curiosity",
    "8": "desire",
    "9": "disappointment",
    "10": "disapproval",
    "11": "disgust",
    "12": "embarrassment",
    "13": "excitement",
    "14": "fear",
    "15": "gratitude",
    "16": "grief",
    "17": "joy",
    "18": "love",
    "19": "nervousness",
    "20": "optimism",
    "21": "pride",
    "22": "realization",
    "23": "relief",
    "24": "remorse",
    "25": "sadness",
    "26": "surprise",
    "27": "neutral",
}

label_strings = []
for key in range(28):
    label_strings.append(label_map[str(key)])


In [ ]:
instance_correctness = True
instance_num = 1
if instance_correctness:
    instance = correct_instances[instance_num-1]
else:
    instance = incorrect_instances[instance_num-1]

inputs = tokenizer.encode(tokenized_datasets["test"]["text"][instance[0]], return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings

# Display attention head view
head_view(attention, tokens)  # Display model view
model_view(attention, tokens, include_layers=[11])  # Display head view

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np

trainer = Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

# Test the model
predictions = trainer.predict(tokenized_datasets["test"])
class_predictions = np.argmax(predictions.predictions, axis=1)
print(
    "Predictions shape: ",
    predictions.predictions.shape,
    "\nLabels shape: ",
    predictions.label_ids.shape,
    "\nClass Predictions: ",
    class_predictions,
)

In [ ]:
def get_instances_by_prediction_correctness(
    dataset, preds, class_preds, correct=True, num_instances=10
):
    assert dataset.num_rows == len(preds.label_ids) == len(class_preds)
    p = np.random.permutation(dataset.num_rows)

    # Collect instances
    instances = []
    i = 0
    while len(instances) < num_instances and i < len(p):
        idx = p[i]

        if (correct and preds.label_ids[idx] == class_preds[idx]) or (
            not correct and preds.label_ids[idx] != class_preds[idx]
        ):
            # Save instance number, tokens, true label, and predicted label (labels as strings)
            instances.append(
                [
                    idx,
                    tokenizer.convert_ids_to_tokens(dataset["text"][idx]),
                    label_map[str(preds.label_ids[idx])],
                    label_map[str(class_preds[idx])],
                ]
            )
            print(instances[-1])

        i += 1

    return instances


correct_instances = get_instances_by_prediction_correctness(
    tokenized_datasets["test"], predictions, class_predictions, correct=True
)

incorrect_instances = get_instances_by_prediction_correctness(
    tokenized_datasets["test"], predictions, class_predictions, correct=False
)

print("Correct Instances: ", correct_instances)
print("Incorrect Instances: ", incorrect_instances)